Website of choice to scrape: 
https://www.petco.com
 

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

import time
import pandas as pd


In [19]:
# getting started and connected

PATH = "C:\Program Files (x86)\web_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get('https://www.petco.com/shop/en/petcostore/category/dog/dog-food/wet-dog-food')

In [20]:
# closes pop-up

popup_close = WebDriverWait(driver, 13).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[14]/div[3]/div/div/div/div[2]/form/div[4]/div[3]/button"))
    )
popup_close.click()

In [21]:
from selenium.common.exceptions import NoSuchElementException

prodLinks=[]
page_count = 1
while True:
    # Increase page_count value on each iteration on +1
    page_count += 1
    
    # extract links
    food1 = driver.find_elements_by_class_name("product-name")
    for tags in food1:
        food2 = tags.find_elements_by_tag_name("a")
        for link in food2:
            prodLinks.append(link.get_attribute("href"))
    try:
        # Clicking on "2" on pagination on first iteration, "3" on second...
        time.sleep(7)
        driver.find_element_by_link_text(str(page_count)).click()
        time.sleep(7)
        driver.find_element_by_link_text(str(page_count)).click()
    except NoSuchElementException:
        # Stop loop if no more page available
        break


In [22]:
len(prodLinks)

698

In [23]:
for i in prodLinks:
    

['https://www.petco.com/shop/en/petcostore/product/wet-noses-wag-soup-turkey-and-pumpkin-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/halo-grain-free-chicken-and-salmon-small-breed-canned-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/hills-science-diet-adult-perfect-weight-hearty-vegetable-and-chicken-stew-canned-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/blue-buffalo-wilderness-trail-toppers-chicken-wild-cuts-dog-food-topper',
 'https://www.petco.com/shop/en/petcostore/product/merrick-lil-plates-small-breed-grain-free-variety-pack-canned-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/wholehearted-grain-free-adult-beef-and-chicken-dinner-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/wholehearted-grain-free-turkey-and-sweet-potato-canned-adult-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/merrick-grain-free-grammys-pot-pie-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/p